# Task description

The file geo_localization.csv contains random geolocations (latitude and longtitude) obtained using two different services, with one taken as the base value (certainly correct) and the other as the prediction result (may be incorrect). The task is to determine and then visualize the prediction error.

Tasks:
+ code implementing the task
+ an .xlsx file containing the original data (without the index) with an additional error column containing the prediction error for each row
+ graphs/charts visualizing the prediction error

In [44]:
# Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [45]:
file_path = os.path.join(os.getcwd(),"data/geo_localization.csv")
df = pd.read_csv(file_path, header=0)

df = df.iloc[:, 1:]

In [46]:
df.head()

,base_lat,base_lon,predicted_lat,predicted_lon
0,52.725344,15.242907,52.725344,15.242905
1,52.238421,20.985096,52.238355,20.985066
2,51.064712,17.006516,51.064600,17.006631
3,50.210517,18.995660,50.210555,18.995702
4,51.052315,17.056584,51.052356,17.056607


## Concepts

I will consider two examples of error between two points. One in the two-dimensional space and second one in the real world using the Earth's radius.

In [47]:
#Variables
base_lat = df['base_lat']
base_lon = df['base_lon']
pred_lat = df['predicted_lat']
pred_lon = df['predicted_lon']

### Euclidean distancer

In [48]:
def euclidean_distance(base_lat, base_lon, pred_lat, pred_lon):
    return np.sqrt((pred_lat - base_lat)**2 + (pred_lon - base_lon)**2)

In [49]:
df['euclidean_distance'] = euclidean_distance(base_lat, base_lon, pred_lat, pred_lon)

### Haversine distance

In [50]:
def haversine_distance(base_lat, base_lon, pred_lat, pred_lon):
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude to radians
    base_lat_rad = np.radians(base_lat)
    base_lon_rad = np.radians(base_lon)
    pred_lat_rad = np.radians(pred_lat)
    pred_lon_rad = np.radians(pred_lon)

    # Calculate differences between latitude and longitude
    diff_lat = pred_lat_rad - base_lat_rad
    diff_lon = pred_lon_rad - base_lon_rad
    # Calculate Haversine distance
    a = np.sin(diff_lat / 2)**2 + np.cos(base_lat_rad) * np.cos(pred_lat_rad) * np.sin(diff_lon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c

    return distance

In [51]:
df['haversine_distance'] = haversine_distance(base_lat, base_lon, pred_lat, pred_lon)

In [52]:
# displaying df head
df.head()

,base_lat,base_lon,predicted_lat,predicted_lon,euclidean_distance,haversine_distance
0,52.725344,15.242907,52.725344,15.242905,0.000002,0.000108
1,52.238421,20.985096,52.238355,20.985066,0.000073,0.007678
2,51.064712,17.006516,51.064600,17.006631,0.000161,0.014860
3,50.210517,18.995660,50.210555,18.995702,0.000057,0.005166
4,51.052315,17.056584,51.052356,17.056607,0.000047,0.004823


In [53]:
# droping NA columns since we don't have error for them
df.replace("", np.nan, inplace=True)

df = df.dropna()

In [54]:
len(df)

1482

### Saving dataframe as xlsx

In [42]:
save_path = "data/geo_localization.xlsx"

df.to_excel(save_path, index=False)